<a href="https://colab.research.google.com/github/160445092/colab_test2/blob/new_sensor/hum_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [2]:

#バターワースフィルタ（バンドストップ）
def bandstop(x, samplerate, fp, fs, gpass, gstop):
    fn = samplerate / 2
    print(fp)                           #ナイキスト周波数
    wp = np.array(fp / 500)                                  #ナイキスト周波数で通過域端周波数を正規化
    ws = np.array(fs / 500)                                #ナイキスト周波数で阻止域端周波数を正規化
    N, Wn = signal.buttord(wp, ws, gpass, gstop)  #オーダーとバターワースの正規化周波数を計算
    b, a = signal.butter(N, Wn, "bandstop")       #フィルタ伝達関数の分子と分母を計算
    y = signal.filtfilt(b, a, x)                  #信号に対してフィルタをかける
    return y         

##関数lowpass定義

In [3]:
import numpy as np
from scipy import signal
import math

#バターワースフィルタ（ローパス）
def lowpass(x, samplerate, fp, fs, gpass, gstop):
    ### 注意　fp<fs じゃないと使えないかも ###

    ### データ補間 2の累乗に合わせる ###
    yososu=len(x)
    print(yososu) #確認
    njo=math.ceil((math.log2(yososu))) #2の何乗？
    hoju=2**njo-1-yososu
    if hoju != 0:
        dainyu=np.full(hoju,x[-1])
        x=np.append(x,dainyu)
    ### データ補間　ここまで ###

    fn = samplerate / 2                           #ナイキスト周波数
    wp = fp / fn                                  #ナイキスト周波数で通過域端周波数を正規化
    ws = fs / fn                                  #ナイキスト周波数で阻止域端周波数を正規化
    N, Wn = signal.buttord(wp, ws, gpass, gstop)  #オーダーとバターワースの正規化周波数を計算
    b, a = signal.butter(N, Wn, "low")            #フィルタ伝達関数の分子と分母を計算
    y = signal.filtfilt(b, a, x)                  #信号に対してフィルタをかける
    y = y[:yososu]                              #はじめに受け取った要素数を返す
    return y 

##関数FFT定義


In [4]:
import numpy as np
import math

def FFT(x, dt):

    ### データ補間 2の累乗に合わせる ###
    yososu=len(x)
    print(yososu) #確認
    njo=math.ceil((math.log2(yososu))) #2の何乗？
    hoju=2**njo-1-yososu
    if hoju != 0:
        dainyu=np.full(hoju,x[-1])
        x=np.append(x,dainyu)
    ### データ補間　ここまで ###

    ### FFT ###

    ## 窓関数
    window_n = np.hanning(len(x))
    # x=x*window_n #周期的な信号のときに使ったほうがよい？
    #print(window_n)

    ## 処理
    F = np.fft.fft(x) # 変換結果
    freq = np.fft.fftfreq(len(x), d=dt) # 周波数
    Amp = np.abs(F/(len(x)/2))   # 振幅

    freq=freq[1:int(len(x)/2)]
    Amp=Amp[1:int(len(x)/2)]

    return freq, Amp 

In [16]:
import numpy as np
import glob
# 配列の情報を見るために用意した関数です。これはcsv読み込みとは関係ありません。
def array_info(x):
    print("配列のshape", x.shape)
    print("配列の要素のデータ型", x.dtype)
    if len(x) >=10:
        print("配列の中身（上から10列）n",x[:10],"n")
    else:
        print("配列の中身n",x,"n")

##全文

In [39]:

""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
"""
#定義ファイル読み込み
import sys
sys.path.append('gdrive/My Drive/Colab Notebooks/EMG_estimation/EMG_hum_filter')
import untitled1_py
"""
""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

import glob
import numpy as np
import csv
import matplotlib.pyplot as plt

""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

#処理設定
#処理するデータのあるフォルダの絶対パスを""内に入力
folder_path = "gdrive/My Drive/Colab Notebooks/EMG_estimation/EMG_hum_filter/test_data"
#サンプリングレート(Hz)
samplingrate = 1000

""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

### 同じフォルダ中のcsvの読み込み ###
list=glob.glob("{}/*.csv".format(folder_path))
#print(list)
if list == []:
  print("リストは空です")
  exit()


for x in list:

    ### ファイル名によって処理を変えたい場合こちらで分岐 ###
    '''
    if 'no1.csv' in x:
        print('no1') ### ファイル名にno1がついたときの処理
    elif '_no2.csv' in x:
        print('no2') ### ファイル名にno2がついたときの処理
    elif '*_no3.csv' in x:
        print('no3') ### ファイル名にno3がついたときの処理
    else:
        print("error")
    '''

    data_tot=[]  #配列を用意
    i=0
    ### CSVの読み込み　自分は行ごとに読み込むのが好み。 ###
    ### 大体のデータロガーは前半に色々な情報書いているため ###
    with open(x,mode="r",encoding="Shift_JIS") as fp:
        csv_reader=csv.reader(fp,delimiter=",") #,区切りでないファイルのときdelimiter
        for row in csv_reader:
            if i>=10:                    #y行目以降を追加。日本語読み込むとエラーになるため
                data_tot.append(row)    #headerは無視
            i=i+1
            print(i)
    print(data_tot) #確認
    data_tot = np.array(data_tot)
    data_tot = data_tot[:,:-1]
    array_info(data_tot)
    data_tot = np.array(data_tot, dtype=float) #要素を型変換。csv_readerは文字列
    #print(data_tot) #確認
    """
    ### 大体のデータの1列目は時間 ###
    dt=data_tot[1,0]-data_tot[0,0]
    samplerate=1/dt
    """
    samplerate = float(samplingrate)
    dt = 1/samplerate

    shori_r=1 #処理したい列
    data=data_tot[:,shori_r]

    ### lowpass ###
    print(data) #確認
    """
    fp = 45      #通過域端周波数[Hz]
    fs = 75      #阻止域端周波数[Hz] fp<fs
    gpass = 3       #通過域端最大損失[dB]
    gstop = 40      #阻止域端最小損失[dB]
    """
    ###バンドストップフィルタの設定###
    fp = np.array([45, 75])         #通過域端周波数[Hz]※ベクトル
    fs = np.array([55, 65])         #阻止域端周波数[Hz]※ベクトル
    gpass = 3                       #通過域端最大損失[dB]
    gstop = 40                      #阻止域端最小損失[dB]
    data_filt = bandstop(data, samplerate, fp, fs, gpass, gstop)

    ###バンドストップフィルタの設定###
    fp = np.array([105, 135])         #通過域端周波数[Hz]※ベクトル
    fs = np.array([115, 125])         #阻止域端周波数[Hz]※ベクトル
    gpass = 3                       #通過域端最大損失[dB]
    gstop = 40                      #阻止域端最小損失[dB]
    data_filt = bandstop(data_filt, samplerate, fp, fs, gpass, gstop)

    ###バンドストップフィルタの設定###
    fp = np.array([165, 195])         #通過域端周波数[Hz]※ベクトル
    fs = np.array([175, 185])         #阻止域端周波数[Hz]※ベクトル
    gpass = 3                       #通過域端最大損失[dB]
    gstop = 40                      #阻止域端最小損失[dB]
    data_filt = bandstop(data_filt, samplerate, fp, fs, gpass, gstop)

    ###バンドストップフィルタの設定###
    fp = np.array([225, 255])         #通過域端周波数[Hz]※ベクトル
    fs = np.array([235, 245])         #阻止域端周波数[Hz]※ベクトル
    gpass = 3                       #通過域端最大損失[dB]
    gstop = 40                      #阻止域端最小損失[dB]
    data_filt = bandstop(data_filt, samplerate, fp, fs, gpass, gstop)

    ###バンドストップフィルタの設定###
    fp = np.array([285, 315])         #通過域端周波数[Hz]※ベクトル
    fs = np.array([295, 305])         #阻止域端周波数[Hz]※ベクトル
    gpass = 3                       #通過域端最大損失[dB]
    gstop = 40                      #阻止域端最小損失[dB]
    data_filt = bandstop(data_filt, samplerate, fp, fs, gpass, gstop)

    ###バンドストップフィルタの設定###
    fp = np.array([345, 375])         #通過域端周波数[Hz]※ベクトル
    fs = np.array([355, 365])         #阻止域端周波数[Hz]※ベクトル
    gpass = 3                       #通過域端最大損失[dB]
    gstop = 40                      #阻止域端最小損失[dB]
    data_filt = bandstop(data_filt, samplerate, fp, fs, gpass, gstop)    

    ###バンドストップフィルタの設定###
    fp = np.array([405, 435])         #通過域端周波数[Hz]※ベクトル
    fs = np.array([415, 425])         #阻止域端周波数[Hz]※ベクトル
    gpass = 3                       #通過域端最大損失[dB]
    gstop = 40                      #阻止域端最小損失[dB]
    data_filt = bandstop(data_filt, samplerate, fp, fs, gpass, gstop)

    ###バンドストップフィルタの設定###
    fp = np.array([465, 495])         #通過域端周波数[Hz]※ベクトル
    fs = np.array([475, 485])         #阻止域端周波数[Hz]※ベクトル
    gpass = 3                       #通過域端最大損失[dB]
    gstop = 40                      #阻止域端最小損失[dB]
    data_filt = bandstop(data_filt, samplerate, fp, fs, gpass, gstop)

    ### FFT ###
    fft, amp = FFT(data, dt)         #処理前データのFFT
    fft2, amp2 = FFT(data_filt, dt)  #処理後データのFFT

    ###グラフ描写のため、データの最大、最小、平均値を算出###
    data_max = max(data)
    data_min = min(data)
    data_filt_max = max(data_filt)
    data_filt_min = min(data_filt)


    ### グラフにしてpngで保存 ###
    ### 保存する紙の広さ ###
    fig=plt.figure(figsize=(18,12)) # 縦横比　(1:1.4) がきれいかも　サイズは(横,縦)

    ax1=fig.add_subplot(2,3,1)   #縦,横,何番目
    ax1.grid()
    ax1.plot(data_tot[:,0],data_tot[:,shori_r],\
             color="red",linestyle="-",\
             label="non-noize") ### ここで横軸の長さをそろえておくと報告書に載せやすい
    ax1.legend() #凡例表示
    print(len(data))
    ax1.set_xlim([0,len(data)]) #軸調整 subplot!!
    ax1.set_ylim((data_min - (data_max*0.1)),(data_max*1.1)) #軸調整 subplot!!

    ax2=fig.add_subplot(2,3,2)
    ax3=ax2.twinx()  #2軸使ってみる
    ax2.grid()
    """
    ax2.plot(data_tot[:,0],data_tot[:,shori_r],\
             color="red",linestyle="-",\
             label="shori_taisho")
    """
    ax3.plot(data_tot[:,0],data_filt[:],\
             color="blue",linestyle="-",\
             label="shori")
    ax2.legend(loc="best") #凡例　左上に
    ax3.legend(loc="best") #凡例　右上に
    
    ax2.set_xlim([0,len(data)]) #軸調整 subplot!!
    ax2.set_ylim((data_filt_min - (data_filt_max*0.1)),(data_filt_max*1.1)) #軸調整 subplot!!
    
    ax3.set_ylim((data_filt_min - (data_filt_max*0.1)),(data_filt_max*1.1)) #軸調整 subplot!!

    ax4=fig.add_subplot(2,3,4)   #縦,横,何番目
    ax4.grid()
    ax4.plot(fft,amp,\
             color="red",linestyle="-",\
             label="fft") ### ここで横軸の長さをそろえておくと報告書に載せやすい

    ax4.legend(loc="upper left") #凡例 左上

    ax5=ax4.twinx() 
    ax5.plot(fft2,amp2,\
             color="blue",linestyle="-",\
             label="fft shori")
    ax5.legend(loc="best") #凡例　右上

    ### 一番右は余白にしてみる ###
    fig.savefig(x[:-3]+"png")
    ### グラフを閉じる。これをしないとメモリーが消費される ###
    plt.close()
    plt.clf()

ストリーミング出力は最後の 5000 行に切り捨てられました。
5046
5047
5048
5049
5050
5051
5052
5053
5054
5055
5056
5057
5058
5059
5060
5061
5062
5063
5064
5065
5066
5067
5068
5069
5070
5071
5072
5073
5074
5075
5076
5077
5078
5079
5080
5081
5082
5083
5084
5085
5086
5087
5088
5089
5090
5091
5092
5093
5094
5095
5096
5097
5098
5099
5100
5101
5102
5103
5104
5105
5106
5107
5108
5109
5110
5111
5112
5113
5114
5115
5116
5117
5118
5119
5120
5121
5122
5123
5124
5125
5126
5127
5128
5129
5130
5131
5132
5133
5134
5135
5136
5137
5138
5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
523

No handles with labels found to put in legend.


<Figure size 432x288 with 0 Axes>